In [1]:
import pandas as pd
import pickle
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification

In [2]:
with open('model.pkl', 'rb') as file_1:
    classification = pickle.load(file_1)

c:\Users\muham\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\muham\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\muham\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.5.2 when using version 1.5.1. This might lead to breaking code or

In [3]:
# Sample data
data = {
    "customer_id": ["CUST023", "CUST024"],
    "tenure": [17, 5],
    "monthly_charges": [2800000, 3400000],
    "total_charges": [47600000, 17000000],
    "contract": ["one year", "month-to-month"],
    "payment_method": ["mailed check", "electronic check"],
    "feedback": ["Reliable and affordable", "Delivery issues multiple times"],
    "topic":["product quality","delivery issues"]
}


In [4]:
# Specify the directory where the model and tokenizer are saved
model_dir = './saved_model/'

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained(model_dir)
model = BertForSequenceClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Tokenize and encode the texts
inputs = tokenizer(
    data['feedback'],
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

# Move inputs to the same device as the model
inputs = {key: val.to(device) for key, val in inputs.items()}

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)


# Map predictions to labels
label_map = {0: 'Negative', 1: 'Positive'}
predicted_labels = [label_map[pred.item()] for pred in predictions]

# Print the results
for i, (text, label) in enumerate(zip(data['feedback'], predicted_labels)):
    print(f"Text: {text}\nPredicted Sentiment: {label}\n")

# Concatenate the sentiment labels into a single column
data['sentiment'] = predicted_labels




Text: Reliable and affordable
Predicted Sentiment: Positive

Text: Delivery issues multiple times
Predicted Sentiment: Negative



In [5]:
# Change Datatype to Dataframe
data = pd.DataFrame(data)

In [6]:
# Do prediction
prediction = classification.predict(data)
prediction

array([False,  True])

In [7]:
# Concatenate the prediction to the data
data['prediction'] = prediction
data

,customer_id,tenure,monthly_charges,total_charges,contract,payment_method,feedback,topic,sentiment,prediction
0,CUST023,17,2800000,47600000,one year,mailed check,Reliable and affordable,product quality,Positive,False
1,CUST024,5,3400000,17000000,month-to-month,electronic check,Delivery issues multiple times,delivery issues,Negative,True
